In [1]:
__imp

Libraries have been loaded


### Load Additional Libraries

In [2]:
import time
import lightgbm as lgb

basepath = os.path.expanduser('../')

## Load Data

In [3]:
%%time

data = pd.read_hdf(os.path.join(basepath, 'data/interim/dataset1/data.h5'))

CPU times: user 3.26 s, sys: 703 ms, total: 3.97 s
Wall time: 3.98 s


## Feature Elimination

- Recursive Feature Elimination

In [5]:
def cv_feedback(Xtr, ytr):
    t0 = time.time()
    
    params = {
    'num_boost_round': 200,
    'early_stopping_rounds': 40,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': .1,
    'metric': 'auc',
    'num_leaves': 7,
    'min_data_in_leaf': 20,
    'nthread': 4,
    'verbose': -1,
    'seed': SEED
    }
    
    num_boost_round       = params['num_boost_round']
    early_stopping_rounds = params['num_boost_round']
    del params['num_boost_round']
    
    ltrain  = lgb.Dataset(Xtr, ytr, feature_name=Xtr.columns.tolist())
    cv_hist = lgb.cv(params, 
                     ltrain, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=None)
    
    cv_hist = pd.DataFrame(cv_hist)
    print('Took: {} seconds'.format(time.time() - t0))
    
    return cv_hist.iloc[-1]['auc-mean']
    
def rfecv(data):
    features = data.columns.drop('TARGET').tolist()
    
    Xtr      = data.loc[:, features]
    ytr      = data.TARGET
    
    max_auc      = cv_feedback(Xtr, ytr)
    feature_copy = features.copy()
    
    for feat in features:
        print('Feature in question: {}'.format(feat))
        
        feature_copy = list(set(feature_copy) - set([feat]))
        loop_auc = cv_feedback(Xtr.loc[:, feature_copy], ytr)
        
        print('Loop AUC: {}'.format(loop_auc))
        print('Max AUC so far: {}'.format(max_auc))
            
        if loop_auc >= max_auc:
            print('Feature removed ')
            max_auc = loop_auc
            
        else:
            print('Adding back feature to feature list.')
            feature_copy = feature_copy + [feat]
    
    return feature_copy

In [6]:
data_sub = data.loc[data.TARGET.notnull(), :].sample(frac=.3)
print(data_sub.shape)

(92253, 559)


In [ ]:
data_sub = data.loc[data.TARGET.notnull(), :]

In [7]:
chosen_features = rfecv(data_sub)

Took: 62.33063626289368 seconds
Feature in question: AMT_ANNUITY
Took: 63.10037899017334 seconds
Loop AUC: 0.7739812768166277
Max AUC so far: 0.7734565102735546
Feature removed 
Feature in question: AMT_CREDIT
Took: 61.99829816818237 seconds
Loop AUC: 0.7736769338262717
Max AUC so far: 0.7739812768166277
Adding back feature to feature list.
Feature in question: AMT_GOODS_PRICE
Took: 62.6885461807251 seconds
Loop AUC: 0.7737301498017408
Max AUC so far: 0.7739812768166277
Adding back feature to feature list.
Feature in question: AMT_INCOME_TOTAL
Took: 62.388842821121216 seconds
Loop AUC: 0.7737200158181141
Max AUC so far: 0.7739812768166277
Adding back feature to feature list.
Feature in question: AMT_REQ_CREDIT_BUREAU_DAY
Took: 63.754438400268555 seconds
Loop AUC: 0.77388378291448
Max AUC so far: 0.7739812768166277
Adding back feature to feature list.
Feature in question: AMT_REQ_CREDIT_BUREAU_HOUR
Took: 61.76818871498108 seconds
Loop AUC: 0.7739812768166277
Max AUC so far: 0.7739812768

Took: 60.54299545288086 seconds
Loop AUC: 0.7748290680551337
Max AUC so far: 0.7748290680551337
Feature removed 
Feature in question: FLAG_DOCUMENT_21
Took: 60.99898290634155 seconds
Loop AUC: 0.7748290680551337
Max AUC so far: 0.7748290680551337
Feature removed 
Feature in question: FLAG_DOCUMENT_3
Took: 59.80300951004028 seconds
Loop AUC: 0.7748290680551337
Max AUC so far: 0.7748290680551337
Feature removed 
Feature in question: FLAG_DOCUMENT_4
Took: 59.832725286483765 seconds
Loop AUC: 0.7748290680551337
Max AUC so far: 0.7748290680551337
Feature removed 
Feature in question: FLAG_DOCUMENT_5
Took: 61.699496030807495 seconds
Loop AUC: 0.7747515368464478
Max AUC so far: 0.7748290680551337
Adding back feature to feature list.
Feature in question: FLAG_DOCUMENT_6
Took: 61.407081604003906 seconds
Loop AUC: 0.7748290680551337
Max AUC so far: 0.7748290680551337
Feature removed 
Feature in question: FLAG_DOCUMENT_7
Took: 61.63926696777344 seconds
Loop AUC: 0.7748290680551337
Max AUC so far:

Took: 59.715346813201904 seconds
Loop AUC: 0.7742074507861535
Max AUC so far: 0.7750700309062682
Adding back feature to feature list.
Feature in question: REG_CITY_NOT_WORK_CITY
Took: 58.72495126724243 seconds
Loop AUC: 0.7750700309062682
Max AUC so far: 0.7750700309062682
Feature removed 
Feature in question: REG_REGION_NOT_LIVE_REGION
Took: 58.614710330963135 seconds
Loop AUC: 0.7750700309062682
Max AUC so far: 0.7750700309062682
Feature removed 
Feature in question: REG_REGION_NOT_WORK_REGION
Took: 58.77262878417969 seconds
Loop AUC: 0.7750700309062682
Max AUC so far: 0.7750700309062682
Feature removed 
Feature in question: SK_ID_CURR
Took: 57.07083868980408 seconds
Loop AUC: 0.7748070990487574
Max AUC so far: 0.7750700309062682
Adding back feature to feature list.
Feature in question: TOTALAREA_MODE
Took: 57.197946548461914 seconds
Loop AUC: 0.7744708078645643
Max AUC so far: 0.7750700309062682
Adding back feature to feature list.
Feature in question: WALLSMATERIAL_MODE
Took: 58.81

Took: 57.22583770751953 seconds
Loop AUC: 0.7743799629395579
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Feature in question: ratio_credit_id_change
Took: 57.097678661346436 seconds
Loop AUC: 0.7739185025494995
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Feature in question: ratio_id_change_age
Took: 60.40682649612427 seconds
Loop AUC: 0.7748581244044205
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Feature in question: diff_id_change_age
Took: 58.040350675582886 seconds
Loop AUC: 0.7740502987566227
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Feature in question: ratio_reg_age
Took: 56.93048691749573 seconds
Loop AUC: 0.7741941222723104
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Feature in question: diff_reg_age
Took: 57.23302984237671 seconds
Loop AUC: 0.7743933782373638
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Featur

Took: 55.86341404914856 seconds
Loop AUC: 0.7740348715677298
Max AUC so far: 0.7753238145526005
Adding back feature to feature list.
Feature in question: median_amt_credit_sum_limit
Took: 57.37819027900696 seconds
Loop AUC: 0.7754205690086535
Max AUC so far: 0.7753238145526005
Feature removed 
Feature in question: mean_cb_credit_annuity
Took: 55.93468499183655 seconds
Loop AUC: 0.7751548961285211
Max AUC so far: 0.7754205690086535
Adding back feature to feature list.
Feature in question: std_cb_credit_annuity
Took: 57.02920341491699 seconds
Loop AUC: 0.7753185728405014
Max AUC so far: 0.7754205690086535
Adding back feature to feature list.
Feature in question: total_credit_sum
Took: 55.46129655838013 seconds
Loop AUC: 0.7746700687074741
Max AUC so far: 0.7754205690086535
Adding back feature to feature list.
Feature in question: num_prev_loans
Took: 55.25858521461487 seconds
Loop AUC: 0.7742255949039719
Max AUC so far: 0.7754205690086535
Adding back feature to feature list.
Feature in q

...

Took: 54.37388324737549 seconds
Loop AUC: 0.7754268368178433
Max AUC so far: 0.7756944646343644
Adding back feature to feature list.
Feature in question: annuity_to_total_income
Took: 55.433735370635986 seconds
Loop AUC: 0.7750034169657078
Max AUC so far: 0.7756944646343644
Adding back feature to feature list.
Feature in question: fg_mode
Took: 55.73897123336792 seconds
Loop AUC: 0.7751521149676961
Max AUC so far: 0.7756944646343644
Adding back feature to feature list.
Feature in question: adjusted_total_income
Took: 55.11854863166809 seconds
Loop AUC: 0.775099077027656
Max AUC so far: 0.7756944646343644
Adding back feature to feature list.
Feature in question: family_housing
Took: 55.19840168952942 seconds
Loop AUC: 0.774546231186814
Max AUC so far: 0.7756944646343644
Adding back feature to feature list.
Feature in question: income_edu_fam_housing
Took: 55.40529227256775 seconds
Loop AUC: 0.7748384502928607
Max AUC so far: 0.7756944646343644
Adding back feature to feature list.
Featur

Took: 52.41999626159668 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: NAME_YIELD_GROUP_0
Took: 53.08054494857788 seconds
Loop AUC: 0.775642214177911
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: NAME_GOODS_CATEGORY_9
Took: 52.98363518714905 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: NAME_GOODS_CATEGORY_7
Took: 53.091570138931274 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: NAME_YIELD_GROUP_3
Took: 52.85525131225586 seconds
Loop AUC: 0.7755598013031585
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: NAME_GOODS_CATEGORY_19
Took: 53.44842076301575 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: PRODUCT_COMBINATION_10
Took: 52.74373149871826 seco

Took: 52.0756516456604 seconds
Loop AUC: 0.7738800507988788
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: mult_amt_payment_sum_delay_installment
Took: 51.53897500038147 seconds
Loop AUC: 0.7741103719482096
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: diff_credit_annuity_age
Took: 52.21492552757263 seconds
Loop AUC: 0.7748924725040018
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: ext_3_age
Took: 51.76699161529541 seconds
Loop AUC: 0.7736285052444387
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: ext_2_age
Took: 51.590009689331055 seconds
Loop AUC: 0.7743570581464634
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: add_rate_ext_2
Took: 52.31866264343262 seconds
Loop AUC: 0.7734795490036909
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.

Took: 51.219828605651855 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: ORGANIZATION_TYPE__36
Took: 51.67457365989685 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: ORGANIZATION_TYPE__37
Took: 51.67763948440552 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: ORGANIZATION_TYPE__38
Took: 52.025885343551636 seconds
Loop AUC: 0.7752267356373644
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: ORGANIZATION_TYPE__39
Took: 51.45277404785156 seconds
Loop AUC: 0.7757312477417047
Max AUC so far: 0.7757312477417047
Feature removed 
Feature in question: ORGANIZATION_TYPE__4
Took: 51.41796135902405 seconds
Loop AUC: 0.7756163275221998
Max AUC so far: 0.7757312477417047
Adding back feature to feature list.
Feature in question: ORGANIZATION_TYPE__40
Took: 51.08071327209473

Took: 51.00223922729492 seconds
Loop AUC: 0.7757360960253964
Max AUC so far: 0.7757360960253964
Feature removed 
Feature in question: NAME_EDUCATION_TYPE__4
Took: 50.46032238006592 seconds
Loop AUC: 0.7757360960253964
Max AUC so far: 0.7757360960253964
Feature removed 
Feature in question: NAME_EDUCATION_TYPE__nan
Took: 51.31384825706482 seconds
Loop AUC: 0.7757360960253964
Max AUC so far: 0.7757360960253964
Feature removed 
Feature in question: NAME_HOUSING_TYPE__0
Took: 50.7494854927063 seconds
Loop AUC: 0.7757146033503727
Max AUC so far: 0.7757360960253964
Adding back feature to feature list.
Feature in question: NAME_HOUSING_TYPE__1
Took: 50.72884440422058 seconds
Loop AUC: 0.7757360960253964
Max AUC so far: 0.7757360960253964
Feature removed 
Feature in question: NAME_HOUSING_TYPE__2
Took: 50.944185972213745 seconds
Loop AUC: 0.7757360960253964
Max AUC so far: 0.7757360960253964
Feature removed 
Feature in question: NAME_HOUSING_TYPE__3
Took: 51.12451243400574 seconds
Loop AUC: 0.

In [ ]:
np.save('../data/interim/dataset1/chosen_features.npy', chosen_features)

In [8]:
np.save('../data/interim/dataset1/chosen_features.npy', chosen_features)